# 广告文案模型定制

使用广告文案数据来定制LLM，使得LLM能够根据服饰描述来自动生成广告文案。DashScope微调定制API的具体说明，可参见[参考文档](https://help.aliyun.com/zh/dashscope/developer-reference/quick-start-16)。

##  1. 安装DashScope Python SDK

In [1]:
!pip install  dashscope

Looking in indexes: https://mirrors.aliyun.com/pypi/simple
Looking in links: https://modelscope.oss-cn-beijing.aliyuncs.com/releases/repo.html

[notice] A new release of pip is available: 23.0 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


1. 请点击链接[模型定制申请链接](https://dashscope.console.aliyun.com/model?spm=a2c4g.11186623.0.0.456e40e9480Mzq&applyModel=llm-fine-tune) 申请模型定制权限

2. 请打开[DashScope官网](https://dashscope.console.aliyun.com/apiKey)，获取你的DashScope API-KEY，并在下面代码块进行替换

In [1]:
# [Note: get your DashScope API key here first: https://dashscope.console.aliyun.com/apiKey]
import dashscope
dashscope.api_key='YOUR-DASHSCOPE-API-KEY'
dashscope.api_key = 'sk-3cef3b0dd2a74e6aaa7d9e8d95efc447'

## 2. 数据处理

### 首先，下载广告文案数据，并解压

In [ ]:
!wget https://dashscope.oss-cn-beijing.aliyuncs.com/samples/fine-tune/AdvertiseGen.tar.gz   && tar -zxf AdvertiseGen.tar.gz

### 查看原数据格式

In [1]:
! cat AdvertiseGen/train.json |head -n 5

{"content": "类型#裤*版型#宽松*风格#性感*图案#线条*裤型#阔腿裤", "summary": "宽松的阔腿裤这两年真的吸粉不少，明星时尚达人的心头爱。毕竟好穿时尚，谁都能穿出腿长2米的效果宽松的裤腿，当然是遮肉小能手啊。上身随性自然不拘束，面料亲肤舒适贴身体验感棒棒哒。系带部分增加设计看点，还让单品的设计感更强。腿部线条若隐若现的，性感撩人。颜色敲温柔的，与裤子本身所呈现的风格有点反差萌。"}
{"content": "类型#裙*风格#简约*图案#条纹*图案#线条*图案#撞色*裙型#鱼尾裙*裙袖长#无袖", "summary": "圆形领口修饰脖颈线条，适合各种脸型，耐看有气质。无袖设计，尤显清凉，简约横条纹装饰，使得整身人鱼造型更为生动立体。加之撞色的鱼尾下摆，深邃富有诗意。收腰包臀,修饰女性身体曲线，结合别出心裁的鱼尾裙摆设计，勾勒出自然流畅的身体轮廓，展现了婀娜多姿的迷人姿态。"}
{"content": "类型#上衣*版型#宽松*颜色#粉红色*图案#字母*图案#文字*图案#线条*衣样式#卫衣*衣款式#不规则", "summary": "宽松的卫衣版型包裹着整个身材，宽大的衣身与身材形成鲜明的对比描绘出纤瘦的身形。下摆与袖口的不规则剪裁设计，彰显出时尚前卫的形态。被剪裁过的样式呈现出布条状自然地垂坠下来，别具有一番设计感。线条分明的字母样式有着花式的外观，棱角分明加上具有少女元气的枣红色十分有年轻活力感。粉红色的衣身把肌肤衬托得很白嫩又健康。"}
{"content": "类型#裙*版型#宽松*材质#雪纺*风格#清新*裙型#a字*裙长#连衣裙", "summary": "踩着轻盈的步伐享受在午后的和煦风中，让放松与惬意感为你免去一身的压力与束缚，仿佛要将灵魂也寄托在随风摇曳的雪纺连衣裙上，吐露出<UNK>微妙而又浪漫的清新之意。宽松的a字版型除了能够带来足够的空间，也能以上窄下宽的方式强化立体层次，携带出自然优雅的曼妙体验。"}
{"content": "类型#上衣*材质#棉*颜色#蓝色*风格#潮*衣样式#polo*衣领型#polo领*衣袖长#短袖*衣款式#拼接", "summary": "想要在人群中脱颖而出吗？那么最适合您的莫过于这款polo衫短袖，采用了经典的polo领口和柔软纯棉面料，让您紧跟时尚潮流。再配合上潮流的蓝色拼接设计，使您

DashScope模型定制化所需的训练数据格式如下：
<!-- ```json
{"prompt": "谁在文艺复兴时期绘制人体", "completion": "文艺复兴时期是一个关于艺术、文化和学术的复兴运动，在这个时期，许多艺术家都绘制了人体。"}
{"prompt": "I need a picture of someone crying.", "completion": "I'm sorry, but as an AI language model, I do not have the ability to display images."}
``` -->
```json
{"text": "\n\nHuman: 谁在文艺复兴时期绘制人体\n\nAssistant: 文艺复兴时期是一个关于艺术、文化和学术的复兴运动，在这个时期，许多艺术家都绘制了人体。"}
{"text": "\n\nHuman: I need a picture of someone crying.\n\nAssistant: I'm sorry, but as an AI language model, I do not have the ability to display images."}
```

我们编写代码，从下载的广告文案数据中，随机抽取200条作为训练数据, 100条作为测试数据，并将其转换为DashScope要求的数据格式。

In [2]:
import json
import random
def data_format_conversion(src_file, dst_file, num):
    with open(src_file, 'r') as infile:
        lines = infile.readlines()

    new_lines = random.choices(lines, k=num)
    with open(dst_file, 'w') as ofile:
        for l in new_lines:
            data = json.loads(l)
            ofile.write(json.dumps({'text': f"\n\nHuman: {data['content']}\n\nAssistant: {data['summary']}"}, ensure_ascii=False) + '\n')
        
raw_train_data = 'AdvertiseGen/train.json'
raw_test_data = 'AdvertiseGen/dev.json'
converted_train_data = 'AdvertiseGen/train_ds.jsonl'
converted_test_data = 'AdvertiseGen/dev_ds.jsonl'
data_format_conversion(raw_train_data, converted_train_data, 200)
data_format_conversion(raw_test_data, converted_test_data, 100)


### 查看转换后的训练集和验证集的数据格式

In [3]:
!cat AdvertiseGen/train_ds.jsonl | head -n 2

{"text": "\n\nHuman: 类型#上衣*颜色#蓝色*图案#条纹*衣样式#衬衫*衣袖型#荷叶袖*衣款式#荷叶边\n\nAssistant: 这款衬衫采用了荷叶袖款式设计。三层荷叶边的融入使衬衫更具浪漫时尚感，非常飘逸甜美，女人味十足。而且蓝色条纹清爽耐看，穿着减龄。"}
{"text": "\n\nHuman: 类型#裙*版型#显瘦*风格#性感*图案#印花*裙长#连衣裙*裙款式#吊带\n\nAssistant: 这款连衣裙采用修身的版型设计，搭配精致的剪裁，穿着更为合体，符合人体工程学。而印花设计，丰富整体的设计感和时尚感，更为抢眼吸睛博人眼球。加上吊带设计，更为性感大方。结合优质的面料，穿着更为亲肤大气。"}
cat: write error: Broken pipe


In [4]:
!cat AdvertiseGen/dev_ds.jsonl | head -n 2

{"text": "\n\nHuman: 类型#裤*版型#宽松*材质#纤维*风格#运动*图案#格子\n\nAssistant: 采用的聚酯纤维面料，具有柔软细滑触感和透气散热特性，让你在行走运动的时候舒适<UNK>。宽松的版型，不会影响肢体自由，让伸展灵活自在。格纹的加入，轻盈立体，具有强的层次感。侧边的织带，柔和裤子的硬朗，突显柔情。"}
{"text": "\n\nHuman: 类型#上衣*版型#宽松*版型#显瘦*材质#棉*图案#线条*衣样式#衬衫\n\nAssistant: 这款衬衫的面料采用纯棉面料，面料手感柔软顺滑，穿着舒适亲肤。版型做了宽松版型设计，藏肉显瘦，适合多数mm驾驭。衣袖直筒设计，修饰纤细手臂线条，下摆包边设计，走线工整，凸显品质精致做工。"}


### 上传数据

In [5]:
from dashscope import File

resp = File.upload(converted_train_data, purpose='fine_tune', description='training data for ad')
if resp['status_code'] != 200:
    print('upload failed')
    print(resp)
else:
    train_file_id = resp['output']['uploaded_files'][0]['file_id']
    print(train_file_id)

538d4c0e-22db-4c89-8236-8229a645765b


In [6]:
resp = File.upload(converted_test_data, purpose='fine_tune', description='validation data for ad')
if resp['status_code'] != 200:
    print('upload failed')
    print(resp)
else:
    val_file_id = resp['output']['uploaded_files'][0]['file_id']
    print(val_file_id)

8c93e840-21c0-41e0-88cd-fc524ab0e151


## 3. 模型训练
### 启动训练

示例代码中我们使用chatglm-6b-v2进行训练，您也可以切换其他模型进行训练，详细的模型列表可以参考[支持定制的模型列表](https://help.aliyun.com/zh/dashscope/developer-reference/list-of-models-that-support-customization/?spm=a2c4g.11186623.0.0.456e40e9480Mzq)

In [7]:
from dashscope import FineTune

model = 'chatglm-6b-v2'

resp = FineTune.call(model=model,
              training_file_ids=[train_file_id],
              validation_file_ids=[val_file_id])
if resp['status_code'] != 200:
    print('finetune job run failed')
    print(resp)
else:
    print('job_id', resp['output']['job_id'])
    print('status', resp['output']['status'])


{'model': 'chatglm-6b-v2', 'training_file_ids': ['538d4c0e-22db-4c89-8236-8229a645765b'], 'validation_file_ids': ['8c93e840-21c0-41e0-88cd-fc524ab0e151'], 'hyper_parameters': {}}
job_id ft-202309201617-7e95
status PENDING


### 查看训练日志，等待训练完成

In [10]:
job_id = resp['output']['job_id']
events = FineTune.stream_events(job_id)
for e in events:
    if e['status_code'] != 200:
        print('finetune job failed')
        break
    elif 'finetune-job succeeded!' in e['output']:
        break
    else:
        print(e['output'])

connect success, now start to follow ft-202309201617-7e95
2023-09-20 16:27:53,436 - INFO - transfer for inference succeeded, start to deliver it for inference
2023-09-20 16:35:36,243 - INFO - finetune-job succeeded


### 训练结束后，获取训练模型名称

In [11]:
resp = FineTune.get(job_id)
if resp['output']['status'] == 'SUCCEEDED':
    finetuned_model = resp['output']['finetuned_output']
    print('finetuned_model:', finetuned_model)
else:
    print('finetune has not been finished')

finetuned_model: chatglm-6b-v2-ft-202309201617-7e95


## 4. 模型部署
部署fine-tuned后的模型到服务，模型的部署需要一定的时间，约10分钟左右， 请耐心等待。

In [13]:
from dashscope import Deployment
import time
resp = Deployment.call(model=finetuned_model, capacity=2)
if resp['status_code'] != 200:
    print('deploy model failed')
    print(resp)
else:
    deployed_model = resp['output']['deployed_model']
    try:
        while True:  # wait for deployment ok.
            resp = dashscope.Deployment.get(deployed_model)
            if resp.status_code == 200:
                if resp.output['status'] in ['PENDING','DEPLOYING']:
                    time.sleep(30)
                    print('Deployment %s is %s' %
                            (deployed_model, resp.output['status']))
                else:
                    print('Deployment: %s status: %s' %
                            (deployed_model, resp.output['status']))
                    break

            else:
                print(resp)
    except Exception:
        print('You can get deployment status via: \
                dashscope deployments.get -d %s' % deployed_model)


{'model_name': 'chatglm-6b-v2-ft-202309201617-7e95', 'capacity': 2, 'suffix': 'aaa'}
Deployment chatglm-6b-v2-ft-202309201617-7e95-aaa is PENDING
Deployment chatglm-6b-v2-ft-202309201617-7e95-aaa is PENDING
Deployment chatglm-6b-v2-ft-202309201617-7e95-aaa is PENDING
Deployment chatglm-6b-v2-ft-202309201617-7e95-aaa is PENDING
Deployment chatglm-6b-v2-ft-202309201617-7e95-aaa is PENDING
Deployment chatglm-6b-v2-ft-202309201617-7e95-aaa is PENDING
Deployment chatglm-6b-v2-ft-202309201617-7e95-aaa is PENDING
Deployment chatglm-6b-v2-ft-202309201617-7e95-aaa is PENDING
Deployment chatglm-6b-v2-ft-202309201617-7e95-aaa is PENDING
Deployment chatglm-6b-v2-ft-202309201617-7e95-aaa is PENDING
Deployment chatglm-6b-v2-ft-202309201617-7e95-aaa is PENDING
Deployment chatglm-6b-v2-ft-202309201617-7e95-aaa is PENDING
Deployment chatglm-6b-v2-ft-202309201617-7e95-aaa is PENDING
Deployment chatglm-6b-v2-ft-202309201617-7e95-aaa is PENDING
Deployment chatglm-6b-v2-ft-202309201617-7e95-aaa is PENDING


## 5. 访问模型服务

当模型部署成功后，状态进入RUNNING。您可以通过提供新部署模型的id，来进行模型的推理调用。以下根据广告数据的格式提供prompt，来获取对应的生成广告文案。

In [16]:
from dashscope import Generation

prompt = '类型#上衣*版型#宽松*版型#显瘦*材质#棉*图案#线条*衣样式#衬衫'

resp = Generation.call(deployed_model, 
                        prompt = prompt,
                        history=[])
if resp.status_code == 200:
    print(resp.output['text']['response'])
else:
    print('request failed')
    print(resp)

这是一款设计感十足的小翻领衬衫，采用了宽松的廓形设计，带来显瘦的视觉效果。加上胸前的立体花朵图案点缀，充满了浪漫的情怀，尽显女性柔美之美。而且，袖口处采用了单边开襟，露出纤细的手臂线条，更显清爽干练。


为了对比finetune的效果， 我们使用相同的输入调用原始的`chatglm-6b-v2`模型

In [17]:
model = 'chatglm-6b-v2'
resp = Generation.call(model, 
                        prompt = prompt,
                        history=[])
if resp.status_code == 200:
    print(resp.output['text']['response'])
else:
    print('request failed')
    print(resp)

根据题目，我们可以得到以下信息：

- 上衣：宽松版型
- 版型：显瘦
- 材质：棉
- 图案：线条
- 衣样式：衬衫

因此，我们可以得出结论：一件宽松的显瘦棉质衬衫的图案是线条。


模型部署后， 通过如下代码（去掉注释）可删除已部署的模型(deployed_model)。

`注意`: 请慎重执行下面语句，执行后新部署的服务将被删除

In [47]:
from dashscope import Deployment
# Deployment.delete(deployed_model)

Deployed model: chatglm-6b-v2-ft-202309111124-c57c delete success
